In [1]:
from zntrack import Node, dvc, config, zn

In [2]:
config.nb_name = "06_named_nodes.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpl60lxlo7/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


# Named Nodes
Named Nodes allow us to use the same Node multiple times in a single graph at e.g. different steps. Therefore, we can pass a `name` argument to the `__init__` of our Node.

<blockquote>Notice that this is one of only very few scenarios where we want to pass an argument directly to the `__init__`</blockquote>

In [5]:
class HelloWorld(Node):
    inputs = zn.params()
    outputs = zn.outs()

    def __init__(self, inputs=None, **kwargs):
        super().__init__(**kwargs)
        self.inputs = inputs

    def run(self):
        self.outputs = self.inputs

In [6]:
HelloWorld(inputs=3).write_graph(run=True)
HelloWorld(name="Test01", inputs=17).write_graph(run=True)
HelloWorld(name="Test02", inputs=42).write_graph(run=True)

2023-02-16 11:25:46,908 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 06_named_nodes.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 11:25:48,227 (WARNING): Running DVC command: 'stage add -n HelloWorld --force ...'


[NbConvertApp] Writing 2605 bytes to 06_named_nodes.py


2023-02-16 11:25:49,545 (INFO): 
2023-02-16 11:25:49,545 (WARNING): Running DVC command: 'repro HelloWorld'
2023-02-16 11:25:51,274 (INFO): 


...

2023-02-16 11:25:53,785 (INFO): 
2023-02-16 11:25:53,785 (WARNING): Running DVC command: 'repro Test01'
2023-02-16 11:25:55,624 (INFO): 


[NbConvertApp] Converting notebook 06_named_nodes.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 11:25:56,914 (WARNING): Running DVC command: 'stage add -n Test02 --force ...'


[NbConvertApp] Writing 2605 bytes to 06_named_nodes.py


2023-02-16 11:25:58,203 (INFO): 
2023-02-16 11:25:58,203 (WARNING): Running DVC command: 'repro Test02'
2023-02-16 11:25:59,991 (INFO): 


In [7]:
!dvc dag

+------------+ 
| HelloWorld | 
+------------+ 
+--------+ 
| Test01 | 
+--------+ 
+--------+ 
| Test02 | 
+--------+ 


We can now also build a Node that depends on multiple of the same Nodes

In [8]:
class FindMaximum(Node):
    deps = zn.deps(
        [
            HelloWorld.load(),
            HelloWorld.load(name="Test01"),
            HelloWorld.load(name="Test02"),
        ]
    )
    maximum = zn.outs()

    def run(self):
        self.maximum = 0
        for node in self.deps:
            if node.outputs > self.maximum:
                self.maximum = node.outputs
                print(f"New maximum found {node.outputs}.")

In [9]:
FindMaximum().write_graph(run=True)

[NbConvertApp] Converting notebook 06_named_nodes.ipynb to script
/data/fzills/miniconda3/envs/zntrack/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


2023-02-16 11:26:01,990 (WARNING): Running DVC command: 'stage add -n FindMaximum --force ...'


[NbConvertApp] Writing 2605 bytes to 06_named_nodes.py


2023-02-16 11:26:03,231 (INFO): 
2023-02-16 11:26:03,231 (WARNING): Running DVC command: 'repro FindMaximum'
2023-02-16 11:26:05,090 (INFO): New maximum found 3.
New maximum found 17.
New maximum found 42.



In [10]:
!dvc dag

+------------+          +--------+          +--------+ 
| HelloWorld |          | Test01 |          | Test02 | 
+------------+**        +--------+       ***+--------+ 
                ***          *        ***              
                   ****     *     ****                 
                       **   *   **                     
                    +-------------+                    
                    | FindMaximum |                    
                    +-------------+                    


Using this combined Node we can e.g. find the maximum of the generated values.

In [11]:
FindMaximum.load().maximum

42

In [12]:
# Running it manually to highlight the print statements
FindMaximum.load().run()

New maximum found 3.
New maximum found 17.
New maximum found 42.


In addition to the introduced classmethod `Node.load(name="nodename")` it is also possible to use `Node["nodename"]`. Note that this only works for `Node["nodename"]` and not for `Node()["nodename"]`. Using this we can also write the following:

In [13]:
print(HelloWorld["Test01"].outputs)
print(HelloWorld["Test01"].node_name)

17
Test01


this is equivalent to the classmethod `load()`. It is also possible to pass a dictionary as kwargs which will be passed to `load(**kwargs)`.

In [14]:
print(HelloWorld.load("Test02").outputs)
print(HelloWorld.load("Test02").node_name)
print(HelloWorld[{"name": "Test02"}].outputs)

42
Test02
42


In [15]:
temp_dir.cleanup()